<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/langchain_vectorizing_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG con Langchain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
# Se instalan las librerías.
!pip install langchain-pinecone==0.2.4
!pip install langchain==0.3.21
!pip install langchain-community==0.3.20
!pip install beautifulsoup4==4.13.3
!pip install tiktoken==0.9.0
!pip install pypdf==5.4.0
!pip install pinecone==6.0.2
!pip install langchain-huggingface==0.1.2

In [ ]:
# Se importan las librerías.
import os
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
import bs4
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from uuid import uuid4
import warnings

warnings.filterwarnings('ignore')

<br>

# Vectorizar información con LangChain

Los **embeddings** crean una **representación vectoria**l de un fragmento de texto. Esto es útil porque nos permite trabajar con el texto en el espacio vectorial y hacer cosas como **búsqueda semántica**, donde buscamos fragmentos de texto que sean más similares dentro de ese espacio.

La clase base `Embeddings` en LangChain proporciona dos métodos: uno para generar embeddings de documentos y otro para consultas. El primero, .`embed_documents`, toma como entrada múltiples textos, mientras que el segundo, .`embed_query`, toma un único texto.

La clase **Embeddings** es una clase diseñada para interactuar con modelos de embeddings de texto. Existen muchos proveedores de modelos de embeddings (OpenAI, Cohere, Hugging Face, etc.), y esta clase está pensada para ofrecer una interfaz estándar para todos ellos.

Un paso fundamental en un sistema RAG es la carga y división de documentos en fragmentos informativos, los cuales serán posteriormente vectorizados y almacenados en una base de datos.

Seguiremos los siguientes pasos:

**Cargar**: Cargar documentos para su división. Esto se hace con Document Loaders (cargadores de documentos).
**Dividir**: Utizaremos Text Splitters (divisores de texto), los cuales dividen documentos grandes en fragmentos más pequeños. Esto es útil tanto para indexar los datos como para pasarlos a un modelo, ya que los fragmentos grandes son más difíciles de buscar y no caben en la ventana de contexto limitada de un modelo.

## Vectorizando un texto con Hugging Face

In [ ]:
# Se prepara el algoritmo a utilizar.
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
text = "Me gusta commer pizza."
query_result = embeddings_model.embed_query(text)
query_result[:3]

Más información en: https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub/

<br>

 # Vectorizando documentos

Utilizaremos el código visto en la [lección anterior](https://colab.research.google.com/drive/1rG_KCvFO6DZN56GruFwESlaxhU09TnSi#scrollTo=8U_GtE_0tuGp), donde cargamos y dividimos información en unidades más pequeñas que llamaremos "documentos".

In [ ]:
# Se carga el documento PDF.
loader = PyPDFLoader('/content/Aviso_de_privacidad.pdf')
pdf_pages = []
async for page in loader.alazy_load():
    pdf_pages.append(page)

# Se carga la información de una página web.

page_url = "https://dialektico.com/cama-ultra-lujosa-para-gatos-dialektiroyal-comfort/"

loader = WebBaseLoader(web_paths=[page_url])
web_info = []
async for doc in loader.alazy_load():
    web_info.append(doc)

### Se generan los fragmentos (documentos a vectorizar)

In [ ]:
# Se instancia el CharacterTextSplitter.
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=1000, chunk_overlap=200
)

# Se divide el texto utilizando el tokenizador tiktoken.
pdf_documents = text_splitter.split_documents(pdf_pages)
web_documents = text_splitter.split_documents(web_info)

Más información en: https://python.langchain.com/docs/concepts/text_splitters/

In [ ]:
# Se unen ambos conjuntos de documentos.
all_documents = pdf_documents + web_documents

# Se extrae sel contenido de texto de cada documento.
all_texts = [doc.page_content for doc in all_documents]

all_texts[0]

In [ ]:
len(all_texts)

In [ ]:
embeddings = embeddings_model.embed_documents(all_texts)

In [ ]:
len(embeddings), len(embeddings[0])

In [ ]:
embeddings[0][:10]

Más información en:
* https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
* https://python.langchain.com/docs/how_to/embed_text/

<br>

# Almacenando vectores con LangChain

**LangChain** proporciona una interfaz estándar para trabajar con **bases de datos vectoriales** (**vector stores**), lo que permite a los usuarios cambiar fácilmente entre distintas implementaciones.

La interfaz consiste en métodos básicos para **escribir**, **eliminar** y **buscar** documentos dentro del almacén vectorial.

Los métodos clave son:

`add_documents`: Agrega una lista de textos al almacén vectorial.

`delete`: Elimina una lista de documentos del almacén vectorial.

`similarity_search`: Busca documentos similares a una consulta dada.


## Almacenando vectores en memoria interna con LangChain

In [ ]:
# Se crea vector store en memoria para almacenar vectores.
vector_store = InMemoryVectorStore(embedding=embeddings_model)

# Se almacenan los vectores.
vector_store.add_documents(documents=all_documents)

<br>

In [ ]:
# Se crea vector store en memoria para almacenar vectores.
vector_store = InMemoryVectorStore(embedding=embeddings_model)

vector_store.add_documents(documents=all_documents, ids=[f"doc_{i+1}" for i in range(len(all_documents))])

In [ ]:
vector_store.store['doc_1']

## Eliminando vectores

In [ ]:
vector_store.delete(ids=["doc_1", "doc_2"])

In [ ]:
vector_store.store['doc_1']

Más información en: https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.in_memory.InMemoryVectorStore.html

<br>

## Buscando vectores

Los vector stores (almacenes vectoriales o bases de datos vectoriales) generan embeddings y almacenan los documentos que se les agregan. Si se proporciona una consulta (query), el almacén vectorial genera el embedding de esa consulta, realiza una búsqueda por similitud sobre los documentos ya embebidos y devuelve los más similares.

Esto implica dos conceptos importantes:

* Debe existir una forma de medir la similitud entre la consulta y cada documento embebido.
* Debe haber un algoritmo eficiente para realizar esta búsqueda de similitud sobre todos los documentos embebidos.

## Métricas de similitud
Una ventaja clave de los vectores es que pueden compararse mediante operaciones matemáticas simples:

* Similitud del coseno (Cosine Similarity): Mide el coseno del ángulo entre dos vectores.
* Distancia euclidiana (Euclidean Distance): Mide la distancia en línea recta entre dos puntos.
* Producto punto (Dot Product): Mide la proyección de un vector sobre otro.

In [ ]:
# Se crea vector store en memoria para almacenar vectores.
vector_store = InMemoryVectorStore(embedding=embeddings_model)

# Se almacenan los vectores.
vector_store.add_documents(documents=all_documents, ids=[f"doc_{i+1}" for i in range(len(all_documents))])

query = "¿Qué incluye la cama ultralujosa para gatos?"
docs = vector_store.similarity_search(query, k=3)

In [ ]:
docs[0]

In [ ]:
query = "¿Por qué la página web DialektiPet utiliza cookies?"
docs = vector_store.similarity_search(query, k=3)

In [ ]:
docs[0]

Más información en: https://python.langchain.com/docs/concepts/vectorstores/

<br>

# Utilizando un vector store con Pinecone

Para poder almacenar los vectores en servicios externos, se puede atender a todas las posibles integraciones de Langchain. En este caso, utilizaremos Pinecone, en donde puedes crear una cuenta gratuita y almacenar un cierto número de vectores con información relacionada.

Para crear tu cuenta, ingresa a aquí: https://www.pinecone.io/

In [ ]:
# Se añade la API key como variable de ambiente desde un secreto en Colab.
os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

In [ ]:
len(embeddings[0])

In [ ]:
pc = Pinecone()
index_name = 'test-langchain'

pc.create_index(
        name='test-langchain',
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [ ]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)

In [ ]:
len(all_documents)

##Indexando documentos

In [ ]:
vector_store.add_documents(documents=all_documents, ids=[str(uuid4()) for _ in range(len(all_documents))])

In [ ]:
query = "¿Qué incluye la cama ultralujosa para gatos?"
docs = vector_store.similarity_search(query, k=3)

<br>

## Buscando vectores con filtro de metadatos

In [ ]:
vector_store.store['doc_11']['metadata']

In [ ]:
query = "¿Qué incluye la cama ultralujosa para gatos?"
docs = vector_store.similarity_search(query, k=3, filter={"source": "https://dialektico.com/cama-ultra-lujosa-para-gatos-dialektiroyal-comfort/"})

Más información en:
*   https://python.langchain.com/docs/integrations/vectorstores/pinecone/
*   https://python.langchain.com/docs/concepts/vectorstores/



In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License